# SSA Demonstration: Denoising, Trend Extraction, and Periodic Components

This notebook demonstrates the capabilities of the MKL-optimized SSA implementation:

1. **Denoising**: Separating signal from noise
2. **Trend Extraction**: Isolating underlying trends
3. **Periodic Extraction**: Identifying and extracting cyclical patterns
4. **Forecasting**: Predicting future values using LRF
5. **MSSA**: Multivariate analysis of correlated series

## Setup

First, ensure `libssa.so` is built:
```bash
source /opt/intel/oneapi/setvars.sh
gcc -shared -fPIC -O3 -o libssa.so ssa_wrapper.c \
    -DSSA_OPT_IMPLEMENTATION -DSSA_USE_MKL \
    -I${MKLROOT}/include -L${MKLROOT}/lib/intel64 -lmkl_rt -lm
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ssa_wrapper import SSA, MSSA

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 10

np.random.seed(42)

---
## 1. Denoising

SSA separates deterministic signal from stochastic noise by concentrating signal energy in the first few singular components.

In [ ]:
# Generate clean signal + noise
N = 500
t = np.linspace(0, 4*np.pi, N)

clean_signal = np.sin(t) + 0.5*np.sin(3*t)  # Two harmonics
noise = 0.5 * np.random.randn(N)
noisy_signal = clean_signal + noise

# Apply SSA
ssa = SSA(noisy_signal, L=100)
ssa.decompose(k=20)

# Reconstruct with first few components (signal)
denoised = ssa.reconstruct([0, 1, 2, 3])  # First 4 components
extracted_noise = ssa.get_noise(noise_start=4)

# Compute metrics
snr_before = 10 * np.log10(np.var(clean_signal) / np.var(noise))
snr_after = 10 * np.log10(np.var(clean_signal) / np.var(denoised - clean_signal))

print(f"SNR before: {snr_before:.1f} dB")
print(f"SNR after:  {snr_after:.1f} dB")
print(f"Improvement: {snr_after - snr_before:.1f} dB")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Original signals
axes[0, 0].plot(t, clean_signal, 'g-', lw=2, label='Clean signal', alpha=0.8)
axes[0, 0].plot(t, noisy_signal, 'b-', lw=0.5, alpha=0.5, label='Noisy signal')
axes[0, 0].set_title('Input: Clean Signal + Noise')
axes[0, 0].legend()
axes[0, 0].set_xlabel('t')

# Denoised result
axes[0, 1].plot(t, clean_signal, 'g-', lw=2, label='Clean signal', alpha=0.8)
axes[0, 1].plot(t, denoised, 'r-', lw=1.5, label='SSA denoised')
axes[0, 1].set_title(f'SSA Denoising (SNR: {snr_before:.1f}→{snr_after:.1f} dB)')
axes[0, 1].legend()
axes[0, 1].set_xlabel('t')

# Residual comparison
axes[1, 0].plot(t, noise, 'b-', lw=0.5, alpha=0.7, label='True noise')
axes[1, 0].plot(t, extracted_noise, 'r-', lw=0.5, alpha=0.7, label='SSA extracted noise')
axes[1, 0].set_title('Noise: True vs Extracted')
axes[1, 0].legend()
axes[1, 0].set_xlabel('t')

# Singular value spectrum
variances = [ssa.variance_explained(i, i) for i in range(min(15, ssa.n_components))]
axes[1, 1].bar(range(len(variances)), variances, color='steelblue', alpha=0.8)
axes[1, 1].axvline(x=3.5, color='red', linestyle='--', label='Signal/Noise cutoff')
axes[1, 1].set_title('Singular Value Spectrum (Variance per Component)')
axes[1, 1].set_xlabel('Component')
axes[1, 1].set_ylabel('Variance Explained')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('ssa_denoising.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 2. Trend Extraction

SSA extracts smooth trends without the lag of moving averages.

In [ ]:
# Generate price-like series: trend + seasonality + noise
N = 600
t = np.arange(N)

trend = 100 + 0.05*t + 5*np.sin(2*np.pi*t/400)  # Slow drift + very slow cycle
seasonal = 3*np.sin(2*np.pi*t/50)  # ~50 period cycle
noise = 1.5*np.random.randn(N)

price = trend + seasonal + noise

# SSA trend extraction
ssa = SSA(price, L=120)
ssa.decompose(k=20)

ssa_trend = ssa.get_trend()  # Component 0

# Compare with moving averages
def moving_average(x, window):
    return np.convolve(x, np.ones(window)/window, mode='same')

ma_50 = moving_average(price, 50)
ma_100 = moving_average(price, 100)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full view
axes[0].plot(t, price, 'gray', lw=0.5, alpha=0.7, label='Price')
axes[0].plot(t, trend, 'g-', lw=2, label='True trend', alpha=0.8)
axes[0].plot(t, ssa_trend, 'r-', lw=2, label='SSA trend')
axes[0].plot(t, ma_50, 'b--', lw=1, label='MA(50)', alpha=0.7)
axes[0].plot(t, ma_100, 'purple', ls='--', lw=1, label='MA(100)', alpha=0.7)
axes[0].set_title('Trend Extraction: SSA vs Moving Averages')
axes[0].legend(loc='upper left')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Price')

# Zoom on turning point (to show lag difference)
zoom_start, zoom_end = 150, 250
axes[1].plot(t[zoom_start:zoom_end], price[zoom_start:zoom_end], 'gray', lw=0.8, alpha=0.7, label='Price')
axes[1].plot(t[zoom_start:zoom_end], trend[zoom_start:zoom_end], 'g-', lw=2.5, label='True trend')
axes[1].plot(t[zoom_start:zoom_end], ssa_trend[zoom_start:zoom_end], 'r-', lw=2, label='SSA trend')
axes[1].plot(t[zoom_start:zoom_end], ma_50[zoom_start:zoom_end], 'b--', lw=1.5, label='MA(50)')
axes[1].plot(t[zoom_start:zoom_end], ma_100[zoom_start:zoom_end], 'purple', ls='--', lw=1.5, label='MA(100)')
axes[1].set_title('Zoomed: Notice MA Lag vs SSA')
axes[1].legend()
axes[1].set_xlabel('Time')

plt.tight_layout()
plt.savefig('ssa_trend.png', dpi=150, bbox_inches='tight')
plt.show()

# Compute RMSE
print(f"RMSE (SSA trend):  {np.sqrt(np.mean((ssa_trend - trend)**2)):.3f}")
print(f"RMSE (MA-50):      {np.sqrt(np.mean((ma_50 - trend)**2)):.3f}")
print(f"RMSE (MA-100):     {np.sqrt(np.mean((ma_100 - trend)**2)):.3f}")

---
## 3. Periodic Component Extraction

SSA identifies periodic signals as pairs of components with nearly equal singular values.

In [ ]:
# Generate multi-frequency signal
N = 500
t = np.arange(N)

trend = 50 + 0.02*t
period1 = 4*np.sin(2*np.pi*t/60)   # Period 60
period2 = 2*np.sin(2*np.pi*t/25)   # Period 25 
period3 = 1*np.sin(2*np.pi*t/12)   # Period 12
noise = 0.8*np.random.randn(N)

signal = trend + period1 + period2 + period3 + noise

# SSA decomposition
ssa = SSA(signal, L=100)
ssa.decompose(k=20)

# Find periodic pairs automatically
pairs = ssa.find_periodic_pairs(max_pairs=10, sv_tol=0.15, wcorr_thresh=0.4)
print(f"Detected periodic pairs: {pairs}")

# Extract components
ssa_trend = ssa.reconstruct([0])

# Try to identify which pair corresponds to which frequency
if len(pairs) >= 1:
    ssa_period1 = ssa.reconstruct(list(pairs[0]))
if len(pairs) >= 2:
    ssa_period2 = ssa.reconstruct(list(pairs[1]))
if len(pairs) >= 3:
    ssa_period3 = ssa.reconstruct(list(pairs[2]))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original signal
axes[0, 0].plot(t, signal, 'b-', lw=0.5, alpha=0.7)
axes[0, 0].plot(t, trend, 'r-', lw=2, label='True trend')
axes[0, 0].set_title('Original Signal (Trend + 3 Periodicities + Noise)')
axes[0, 0].legend()

# W-correlation matrix
W = ssa.wcorr_matrix()
im = axes[0, 1].imshow(np.abs(W[:12, :12]), cmap='RdBu_r', vmin=0, vmax=1)
axes[0, 1].set_title('W-Correlation Matrix (|ρ|)')
axes[0, 1].set_xlabel('Component')
axes[0, 1].set_ylabel('Component')
plt.colorbar(im, ax=axes[0, 1], shrink=0.8)

# Extracted periodicities
axes[1, 0].plot(t, period1, 'g-', lw=2, alpha=0.5, label='True P=60')
if len(pairs) >= 1:
    axes[1, 0].plot(t, ssa_period1, 'r-', lw=1.5, label=f'SSA pair {pairs[0]}')
axes[1, 0].plot(t, period2, 'b-', lw=2, alpha=0.5, label='True P=25')
if len(pairs) >= 2:
    axes[1, 0].plot(t, ssa_period2, 'orange', lw=1.5, label=f'SSA pair {pairs[1]}')
axes[1, 0].set_title('Periodic Component Extraction')
axes[1, 0].legend(ncol=2)
axes[1, 0].set_xlim(0, 200)  # Zoom for clarity

# Singular value spectrum with pairs highlighted
n_show = 15
sigmas = [np.sqrt(ssa.variance_explained(i, i) * 100) for i in range(n_show)]  # Relative scale
colors = ['steelblue'] * n_show
for p in pairs:
    for idx in p:
        if idx < n_show:
            colors[idx] = 'red'

axes[1, 1].bar(range(n_show), sigmas, color=colors, alpha=0.8)
axes[1, 1].set_title('Singular Values (red = periodic pairs)')
axes[1, 1].set_xlabel('Component')
axes[1, 1].set_ylabel('Relative Magnitude')

plt.tight_layout()
plt.savefig('ssa_periodic.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Forecasting with LRF

SSA uses the Linear Recurrence Formula to extrapolate the signal.

In [ ]:
# Generate signal with trend + seasonality
N = 400
N_forecast = 100
t_full = np.arange(N + N_forecast)

# True signal (known for entire range)
true_trend = 100 + 0.03*t_full
true_seasonal = 5*np.sin(2*np.pi*t_full/50)
true_signal = true_trend + true_seasonal

# Observed signal (only first N points, with noise)
noise = 1.0*np.random.randn(N)
observed = true_signal[:N] + noise

# Fit SSA on observed data
ssa = SSA(observed, L=80)
ssa.decompose(k=20)

# Forecast
signal_components = [0, 1, 2]  # Trend + first periodic pair
full_reconstruction = ssa.forecast_full(signal_components, N_forecast)

reconstruction = full_reconstruction[:N]
forecast = full_reconstruction[N:]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full view
axes[0].plot(t_full[:N], observed, 'gray', lw=0.5, alpha=0.5, label='Observed')
axes[0].plot(t_full, true_signal, 'g-', lw=2, alpha=0.7, label='True signal')
axes[0].plot(t_full[:N], reconstruction, 'b-', lw=1.5, label='SSA reconstruction')
axes[0].plot(t_full[N:], forecast, 'r-', lw=2, label='SSA forecast')
axes[0].axvline(x=N, color='black', linestyle='--', alpha=0.5)
axes[0].fill_between([N, N+N_forecast], -10, 200, alpha=0.1, color='red')
axes[0].set_title('SSA Forecasting')
axes[0].legend(loc='upper left')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Value')
axes[0].set_ylim(90, 130)

# Forecast zoom
t_forecast = t_full[N:]
axes[1].plot(t_forecast, true_signal[N:], 'g-', lw=2.5, label='True signal')
axes[1].plot(t_forecast, forecast, 'r-', lw=2, label='SSA forecast')
axes[1].fill_between(t_forecast, true_signal[N:], forecast, alpha=0.3, color='red')
axes[1].set_title('Forecast Region (Zoomed)')
axes[1].legend()
axes[1].set_xlabel('Time')

plt.tight_layout()
plt.savefig('ssa_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

# Compute forecast error
forecast_rmse = np.sqrt(np.mean((forecast - true_signal[N:])**2))
forecast_mape = np.mean(np.abs((forecast - true_signal[N:]) / true_signal[N:])) * 100
print(f"Forecast RMSE: {forecast_rmse:.3f}")
print(f"Forecast MAPE: {forecast_mape:.2f}%")

---
## 5. MSSA: Multivariate Analysis

MSSA extracts common factors from correlated time series.

In [ ]:
# Simulate 4 correlated "sector ETFs"
N = 400
t = np.arange(N)

# Common market factor
market = 100 + 0.05*t + 8*np.sin(2*np.pi*t/100)

# Sector-specific factors
sector_betas = [1.0, 0.8, 1.2, 0.6]  # Market exposure
sector_offsets = [0, 20, -10, 40]  # Base level
sector_noise = [1.5, 2.0, 1.8, 2.5]  # Idiosyncratic vol

# Generate series
X = np.zeros((4, N))
for i in range(4):
    X[i] = sector_offsets[i] + sector_betas[i] * market + sector_noise[i] * np.random.randn(N)

# Apply MSSA
mssa = MSSA(X, L=80)
mssa.decompose(k=15)

print(f"Variance explained by first 3 components: {mssa.variance_explained(0, 2):.1%}")

In [ ]:
# Extract common factor from each series
common = mssa.reconstruct_all([0])  # First component = market

# Series contributions
contrib = mssa.series_contributions()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original series
colors = ['blue', 'orange', 'green', 'red']
for i in range(4):
    axes[0, 0].plot(t, X[i], color=colors[i], lw=0.5, alpha=0.7, label=f'Series {i}')
axes[0, 0].set_title('Original Series (4 Correlated ETFs)')
axes[0, 0].legend()
axes[0, 0].set_xlabel('Time')

# Extracted common factor vs true market
axes[0, 1].plot(t, market, 'k-', lw=2, label='True market factor', alpha=0.7)
for i in range(4):
    # Scale common factor to match market (different betas)
    scaled = common[i] - np.mean(common[i]) + np.mean(market)
    axes[0, 1].plot(t, common[i], color=colors[i], lw=1, alpha=0.5, label=f'Common from S{i}')
axes[0, 1].set_title('Extracted Common Factor vs True Market')
axes[0, 1].legend(ncol=2)
axes[0, 1].set_xlabel('Time')

# Series contributions heatmap
im = axes[1, 0].imshow(contrib[:, :8], aspect='auto', cmap='YlOrRd')
axes[1, 0].set_title('Series Contributions to Components')
axes[1, 0].set_xlabel('Component')
axes[1, 0].set_ylabel('Series')
axes[1, 0].set_yticks(range(4))
axes[1, 0].set_yticklabels([f'S{i}' for i in range(4)])
plt.colorbar(im, ax=axes[1, 0], shrink=0.8)

# Residuals (idiosyncratic)
residuals = mssa.reconstruct_all(list(range(2, 10)))  # Components 2+ = idiosyncratic
for i in range(4):
    axes[1, 1].plot(t, residuals[i], color=colors[i], lw=0.5, alpha=0.7, label=f'Residual {i}')
axes[1, 1].set_title('Idiosyncratic Residuals (should be uncorrelated)')
axes[1, 1].legend()
axes[1, 1].set_xlabel('Time')

plt.tight_layout()
plt.savefig('mssa_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

# Compute residual correlations
print("\nResidual correlations (should be low):")
for i in range(4):
    for j in range(i+1, 4):
        corr = np.corrcoef(residuals[i], residuals[j])[0, 1]
        print(f"  Series {i} vs {j}: {corr:.3f}")

---
## 6. Performance Benchmark

Compare decomposition methods and measure throughput.

In [ ]:
import time

# Generate large signal
N = 5000
L = 1000
k = 50

signal = np.cumsum(np.random.randn(N)) + 10*np.sin(2*np.pi*np.arange(N)/100)

print(f"Signal: N={N}, L={L}, k={k}")
print(f"Hankel matrix: {L} × {N-L+1} = {L*(N-L+1):,} elements")
print()

# Benchmark randomized SVD
start = time.perf_counter()
ssa = SSA(signal, L=L)
ssa.decompose(k=k, method='randomized')
elapsed = time.perf_counter() - start
print(f"Randomized SVD: {elapsed*1000:.1f} ms")

# Benchmark block method
start = time.perf_counter()
ssa2 = SSA(signal, L=L)
ssa2.decompose(k=k, method='block')
elapsed = time.perf_counter() - start
print(f"Block method:   {elapsed*1000:.1f} ms")

# Variance explained should be similar
print(f"\nVariance explained (first 10):")
print(f"  Randomized: {ssa.variance_explained(0, 9):.4f}")
print(f"  Block:      {ssa2.variance_explained(0, 9):.4f}")

---
## Summary

This MKL-optimized SSA implementation provides:

| Feature | Method | Typical Use |
|---------|--------|-------------|
| **Denoising** | `reconstruct([0..k])` | Clean price data |
| **Trend** | `get_trend()` | Identify direction |
| **Periodicity** | `find_periodic_pairs()` | Seasonality detection |
| **Forecast** | `forecast()` | Price prediction |
| **Multivariate** | `MSSA` | Pairs trading, factor extraction |

Performance: ~20-40ms for N=5000, L=1000, k=50 using randomized SVD.